In [17]:
import os
from pyhdf.SD import SD, SDC
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pathlib import Path
from scipy.ndimage import label

# Constants
THRESH_BT = 233  # Brightness temperature threshold (K)
MIN_PIXELS = 600  # Minimum number of pixels to classify as MCS

# Function to read MODIS HDF files and extract relevant datasets (latitude, longitude, temperature)
def read_modis_hdf(hdf_files):
    modis_data = []
    
    for file in hdf_files:
        try:
            hdf = SD(file, SDC.READ)  # Open the HDF file
            
            # Extract relevant datasets
            latitude = hdf.select('Latitude')[:]
            longitude = hdf.select('Longitude')[:]
            brightness_temperature = hdf.select('Brightness_Temperature')[:]
            scan_start_time = hdf.select('Scan_Start_Time')[:]  # Assuming this field exists for time
            
            # Convert the time from seconds since 1993-01-01 to datetime
            base_time = datetime(1993, 1, 1)
            time = [base_time + timedelta(seconds=t) for t in scan_start_time.flatten()]
            
            # Store the extracted data
            modis_data.append({
                'latitude': latitude,
                'longitude': longitude,
                'brightness_temperature': brightness_temperature,
                'time': time  # Add time to the data
            })
        except Exception as e:
            print(f"Error processing file {file}: {e}")
    
    return modis_data

# Function to process granules (read files, extract data, and write CSV)
def process_granules(granules, start_date, end_date, output_csv):
    records = []
    
    for f in granules:
        fn = Path(f).name
        parts = fn.split(".")
        year = int(parts[1][1:5])
        doy  = int(parts[1][5:])
        hhmm = parts[2]
        hour   = int(hhmm[:2])
        minute = int(hhmm[2:])
        
        # Build a datetime object based on year, day of year, hour, and minute
        date0 = datetime(year, 1, 1) + timedelta(days=doy-1, hours=hour, minutes=minute)

        try:
            # Open the granule
            sd = SD(f, SDC.READ)
            ctt_raw = sd.select('Cloud_Top_Temperature')[:].astype(float)
            lat = sd.select('Latitude')[:]
            lon = sd.select('Longitude')[:]
            attrs = sd.select('Cloud_Top_Temperature').attributes()
            ctt = (ctt_raw - attrs['add_offset']) * attrs['scale_factor']
            ctt = np.ma.masked_where(ctt_raw == attrs['_FillValue'], ctt)  # Mask invalid values

            # Check if data is empty or invalid
            if np.all(np.isnan(ctt)):
                print(f"Skipping granule {f} because data is invalid.")
                continue  # Skip invalid granules

            # Wrap longitude to the range [-180, 180]
            lon_wrapped = (lon + 180.) % 360. - 180.
            cold = ctt <= THRESH_BT
            blobs, nlab = label(cold)

            for lab in range(1, nlab + 1):
                mask = blobs == lab
                if mask.sum() < MIN_PIXELS:
                    continue  # Skip blobs that don't meet the size requirement

                # Calculate average latitude and longitude for the blob
                clat = float(lat[mask].mean())
                clon = float(np.degrees(np.arctan2(np.sin(np.radians(lon_wrapped[mask])).mean(),
                                                   np.cos(np.radians(lon_wrapped[mask])).mean())))
                
                # Append the detected MCS data to the records list
                records.append({
                    "time": date0,
                    "granule": fn,
                    "lon": clon,
                    "lat": clat,
                    "MCS_minBT": float(ctt[mask].min()),
                    "MCS_avgBT": float(ctt[mask].mean()),
                    "MCS_size": int(mask.sum()),
                })
        except Exception as e:
            print(f"Error processing granule {f}: {e}")
    
    # Write the records to a CSV file
    df = pd.DataFrame(records)
    df.to_csv(output_csv, index=False)
    print(f"Wrote {len(df)} rows to {output_csv}")
    return df

# List of available granules (as provided by you)
granules = [
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002244.0045.061.2018004074116.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002244.0220.061.2018004075640.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002244.0400.061.2018004075305.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002244.0540.061.2018004074751.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002244.0715.061.2018004075605.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002245.0125.061.2018004080039.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002245.0300.061.2018004081337.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002245.0440.061.2018004075606.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002245.0630.061.2018004075900.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002246.0035.061.2018004081156.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002246.0215.061.2018004082114.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002246.0355.061.2018004081219.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002246.0535.061.2018004082029.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002246.0710.061.2018004081945.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002247.0110.061.2018004083640.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002247.0250.061.2018004083644.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002247.0430.061.2018004082439.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002247.0605.061.2018004082100.hdf',
    '/Users/fadiya/Documents/cycone/data/downloads/MYD06_L2.A2002247.0745.061.2018004083222.hdf'
]

# Define the date range for processing
start_date = "2002-09-01"
end_date = "2002-09-04"
output_csv = "mcs_full.csv"

# Process the granules and output the MCS data
mcs_df = process_granules(granules, start_date, end_date, output_csv)

# Show the first few rows of the processed data
print(mcs_df.head())


/var/folders/f2/_6r3zjz171xcs8vgzgkdppc00000gn/T/ipykernel_85106/717112647.py:96: UserWarning: Warning: converting a masked element to nan.
  "MCS_minBT": float(ctt[mask].min()),
/var/folders/f2/_6r3zjz171xcs8vgzgkdppc00000gn/T/ipykernel_85106/717112647.py:97: UserWarning: Warning: converting a masked element to nan.
  "MCS_avgBT": float(ctt[mask].mean()),


Wrote 115 rows to mcs_full.csv
                 time                                       granule  \
0 2002-09-01 00:45:00  MYD06_L2.A2002244.0045.061.2018004074116.hdf   
1 2002-09-01 00:45:00  MYD06_L2.A2002244.0045.061.2018004074116.hdf   
2 2002-09-01 00:45:00  MYD06_L2.A2002244.0045.061.2018004074116.hdf   
3 2002-09-01 00:45:00  MYD06_L2.A2002244.0045.061.2018004074116.hdf   
4 2002-09-01 00:45:00  MYD06_L2.A2002244.0045.061.2018004074116.hdf   

          lon        lat   MCS_minBT   MCS_avgBT  MCS_size  
0 -166.064102  10.966345         NaN         NaN       670  
1 -172.369415  10.166039         NaN         NaN      1263  
2 -178.838989  11.009789  232.129995  232.129995      1042  
3 -165.777390  16.864054  198.789996  222.299237      3341  
4 -178.480469  20.104568  194.589996  217.285998     13885  
